# Model Deployment on Vertex AI with Tensorflow

Contributors: michaelmenzel@google.com

In [11]:
!pip install -q --user --no-warn-conflicts google-cloud-aiplatform

In [11]:
from google.cloud import aiplatform

aiplatform.init(location='europe-west4')

## Model Training

In [12]:
%%writefile tf_train.py

import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

import argparse
import joblib
import os

parser = argparse.ArgumentParser()
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Epocsh to run the model fit function.')
args = parser.parse_args()

iris_data, iris_info = tfds.load('iris', split='train', with_info=True)

iris_ds = iris_data.map(lambda x: (x['features'], tf.one_hot(x['label'], iris_info.features['label'].num_classes))).batch(5)
iris_train = iris_ds.skip(15)
iris_test = iris_ds.take(15)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=iris_info.features['features'].shape),
    tf.keras.layers.Dense(iris_info.features['label'].num_classes, activation='softmax')  
])
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.fit(iris_train, epochs=args.epochs, verbose=2)

model.evaluate(iris_test, verbose=2)
model.save(os.getenv("AIP_MODEL_DIR"))

Overwriting tf_train.py


In [13]:
vertex_ai_job = aiplatform.CustomTrainingJob(
    display_name='tf-iris-classifier-training',
    script_path='tf_train.py',
    container_uri='europe-docker.pkg.dev/vertex-ai/training/tf-cpu.2-6:latest',
    staging_bucket='gs://sandbox-michael-menzel-training-europe-west4/tf-iris-classifier-training/',
    requirements=['tensorflow-datasets'],
    model_serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest',
)

vertex_ai_model = vertex_ai_job.run(
    replica_count=1, 
    machine_type='e2-standard-8',
    args=[
        f"--epochs=100",
    ]
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://sandbox-michael-menzel-training-europe-west4/tf-iris-classifier-training/aiplatform-2022-01-21-22:00:19.659-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://sandbox-michael-menzel-training-europe-west4/tf-iris-classifier-training/aiplatform-custom-training-2022-01-21-22:00:19.860 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/855703720409694208?project=928871478446
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/928871478446/locations/europe-west4/trainingPipelines/855703720409694208 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/1676484752497967104?project=928871478446
INFO:google

In [14]:
vertex_ai_endpoint = vertex_ai_model.deploy(
    deployed_model_display_name='tf-iris-classifier-endpoint',
    traffic_split={"0": 100},
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/928871478446/locations/europe-west4/endpoints/345035544889131008/operations/8028437093438980096
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/928871478446/locations/europe-west4/endpoints/345035544889131008
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/928871478446/locations/europe-west4/endpoints/345035544889131008')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/928871478446/locations/europe-west4/endpoints/345035544889131008
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/345035544889131008/operations/5159644130803974144
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/92887147844

In [15]:
vertex_ai_endpoint

resource name: projects/928871478446/locations/europe-west4/endpoints/345035544889131008

In [16]:
aiplatform.models.Endpoint('projects/928871478446/locations/europe-west4/endpoints/345035544889131008')

resource name: projects/928871478446/locations/europe-west4/endpoints/345035544889131008

In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds

iris_data, iris_info = tfds.load('iris', split='train', with_info=True)
iris_ds = iris_data.map(lambda x: (x['features'], tf.one_hot(x['label'], iris_info.features['label'].num_classes))).batch(5)
iris_train = iris_ds.skip(15)
iris_test = iris_ds.take(15)

precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

for X_test, y_test in iris_test.as_numpy_iterator():
    vertex_ai_predicted = vertex_ai_endpoint.predict(X_test.tolist())
    print(f'categorical accuracy: {tf.keras.metrics.categorical_accuracy(y_test, vertex_ai_predicted.predictions)}')
    precision.update_state(y_test, vertex_ai_predicted.predictions)
    recall.update_state(y_test, vertex_ai_predicted.predictions)

print(f'precision: {precision.result()}')
print(f'recall: {precision.result()}')

INFO:absl:Load dataset info from /home/jupyter/tensorflow_datasets/iris/2.0.0
INFO:absl:Reusing dataset iris (/home/jupyter/tensorflow_datasets/iris/2.0.0)
INFO:absl:Constructing tf.data.Dataset iris for split train, from /home/jupyter/tensorflow_datasets/iris/2.0.0
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]
categorical accuracy: [0. 1. 1. 1. 1.]
categorical accuracy: [1. 1. 1. 1. 1.]


2021-10-21 12:56:53.132398: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


precision: 0.9866666793823242
recall: 0.9866666793823242


## Cleanup

In [11]:
vertex_ai_endpoint.delete(force=True)

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/928871478446/locations/europe-west4/endpoints/8187420977257250816
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/8187420977257250816/operations/2105935302609600512
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/928871478446/locations/europe-west4/endpoints/8187420977257250816
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/928871478446/locations/europe-west4/endpoints/8187420977257250816
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/928871478446/locations/europe-west4/operations/7108308588711378944
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/928871478446/locations/europe-west4/endpoints/8187420977257250816


In [12]:
vertex_ai_model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/928871478446/locations/europe-west4/models/7421386828139200512
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/928871478446/locations/europe-west4/operations/1586895445555150848
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/928871478446/locations/europe-west4/models/7421386828139200512
